In [1]:
#import relevant libraries 
import pandas as pd
import plotly.express as px
import panel as pn
#import hvplot.pandas
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path

import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import os
%matplotlib inline
import nltk
nltk.download('vader_lexicon')
from datetime import datetime, timedelta,date
from pandas import DataFrame
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()



[nltk_data] Error loading vader_lexicon: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1076)>


In [2]:
## Get Covid Case Count Data
#using url for covid 19 api united states case counts confirmed
request_url = "https://api.covid19api.com/total/dayone/country/united-states/status/confirmed"

def get_case_count(request_url):
    # Submit request and format output
    response_data = requests.get(request_url).json()
    states_json = json.dumps(response_data, indent=4)
    case_count_df = pd.read_json(request_url)
    #clean data
    case_count_df.drop(columns = ['CountryCode', 'Province', 'City', 'CityCode', 'Lat','Lon'], inplace = True)
    #update date time stamp to just date
    case_count_df['Date']=case_count_df['Date'].dt.date
    case_count_df.set_index('Date', inplace = True)
    #add percent change column
    case_pct_change = case_count_df['Cases'].pct_change()
    case_count_df['Pct_Change']= case_pct_change
    #add 7 day rolling moving average
    case_rolling = case_count_df['Cases'].rolling(window = 7).mean()
    case_count_df['7day_Rolling'] = case_rolling
    #drop nulls
    case_count_df.dropna(inplace = True)
    return case_count_df

In [3]:
case_count_df=get_case_count(request_url)


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1076)>

In [4]:
## Get Covid 19 News Data
# Read api key environment variable for news api
load_dotenv()
newsapi_key = os.getenv("newsapikey")

import newsapi
from newsapi import NewsApiClient

#get news articles on covid 19
# Create a newsapi client
newsapi = NewsApiClient(api_key=newsapi_key)
    
    

In [5]:
## Get Market Data for S&P500 
#engage API keys by activating .env file for Alpaca Api
load_dotenv()
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

def get_SP500_data(api):
    #load in historical data for S&P 500
    sp500_df = api.alpha_vantage.historic_quotes('SPY', adjusted=True, output_format='pandas')
    
    #Clean Data
    #Select the column we need,  "adjusted close", and drop the others
    sp500_df = sp500_df['5. adjusted close']
    #Sort earliest to latest so that .pct_change() function works right.
    sp500_df.sort_index(inplace=True, ascending=True)

    #create a dataframe column for the daily returns (pct_change) values and concat with SP500 close
    returns_df = sp500_df.pct_change()
    all_returns_df = pd.concat([sp500_df, returns_df], axis="columns", join="inner")

    #Change column names to avoid confusion
    columns = ['S&P 500 close','S&P 500 Daily Returns']
    all_returns_df.columns = columns

    # Drop nulls
    all_returns_df.dropna(inplace=True)

    #drop duplicates
    all_returns_df.drop_duplicates(inplace=True) 
    return all_returns_df

In [6]:
all_returns_df=get_SP500_data(api)

In [7]:
def covid_case_count(case_count_df):
    case_count_signals_df = case_count_df.loc[:, ['Cases']].copy()
    short_window = 7
    long_window = 30

    case_count_signals_df['7day % Change'] = case_count_signals_df['Cases'].pct_change().rolling(short_window).mean()
    case_count_signals_df['30day % Change'] = case_count_signals_df['Cases'].pct_change().rolling(long_window).mean()

    case_count_signals_df['Case Count Signal'] = 0.0
    case_count_signals_df['Case Count Signal'][short_window:] = np.where(case_count_signals_df['7day % Change'][short_window:] > case_count_signals_df['30day % Change'][short_window:], 1.0, -1.0)

    return case_count_signals_df
    

In [8]:
def covid_sentiment(newsapi):
    daily_signal_dict={}
    daily_sentiment_dict={}

    for i in range(30):

        # Set start and end datetimes of for 1 days of news
        end_date = datetime.now()
        start_date = end_date + timedelta(-i)
        end_date=end_date.strftime("%Y-%m-%d")
        start_date=start_date.strftime("%Y-%m-%d")

        # Fetch the Covid19 news articles
        covid19_news = newsapi.get_everything(
        from_param=start_date,
        to=end_date,
        q="Covid 19",
        language="en",
        page_size=100,
        sort_by="relevancy"
        )
    

        #covid-19 sentiment list to dataframe
        covid19_sentiment_list=[]
        for article in covid19_news["articles"]:        
            try: 
                text = article["content"]
                sentiment = analyzer.polarity_scores(text)
                compound = sentiment["compound"]
                pos = sentiment["pos"]
                neu = sentiment["neu"]
                neg = sentiment["neg"]
                scores={"Compound":compound, "Negative":neg, "Neutral":neu, "Positive":pos, "text":text}
                covid19_sentiment_list.append(scores)
        
            except: 
                pass

        covid19_sentiment_df=pd.DataFrame(covid19_sentiment_list)
    

        # Describe the  Sentiment Related to Covid19/Coronavirus
        sentiment_df=covid19_sentiment_df.describe()


        #determine trading signal value based on covid news sentiment 
        if sentiment_df['Neutral'][1] > (sentiment_df['Positive'][1] and sentiment_df['Negative'][1]):
            sentiment_signal=0.0
            sentiment='Neutral'
        elif sentiment_df['Negative'][1] > (sentiment_df['Positive'][1] and sentiment_df['Neutral'][1]):
            sentiment_signal=-1.0
            sentiment='Negative'
        elif sentiment_df['Positive'][1] > (sentiment_df['Negative'][1] and sentiment_df['Netural'][1]):
            sentiment_signal=1.0
            sentiment='Positive'

        daily_signal_dict.update({start_date:sentiment_signal})
        daily_sentiment_dict.update({start_date:sentiment})
   
    #convert dictionary to dataframe     
    daily_signal_df=DataFrame.from_dict(daily_signal_dict,orient='index',columns=['Sentiment Signal'])
    daily_sentiment_df=DataFrame.from_dict(daily_sentiment_dict,orient='index',columns=['Sentiment'])

    sentiment_signal_df=pd.concat([daily_signal_df,daily_sentiment_df],join='inner', axis=1)
    sentiment_signal_df.sort_index(inplace=True, ascending=True)
    return sentiment_signal_df


In [21]:
def get_EMA(all_returns_df):
    # Grab just the `date` and `close` from the dataset
    sp500_signals = all_returns_df.loc[:, ["S&P 500 close"]].copy()

    # Set the short window and long windows
    short_window = 50
    long_window = 100

    # Generate the short and long exponential moving averages (50 and 100 days, respectively)
    sp500_signals["EWM50"] = sp500_signals['S&P 500 close'].ewm(span=short_window).mean()
    sp500_signals["EWM100"] = sp500_signals['S&P 500 close'].ewm(span=long_window).mean()
    sp500_signals["Signal"] = 0.0

    # Generate the trading signal 0 or 1,
    # where 0 is when the EWM50 is under the EWM100, and
    # where 1 is when the EWM50 is higher (or crosses over) the SMA100
    sp500_signals["Signal"][short_window:] = np.where(
    sp500_signals["EWM50"][short_window:] > sp500_signals["EWM100"][short_window:], 1.0, 0.0
)
    # Calculate the points in time at which a position should be taken, 1 or -1
    sp500_signals["Entry/Exit"] = sp500_signals["Signal"].diff()

    sp500_signals.head(10)# Grab just the `date` and `close` from the dataset
    sp500_signals = all_returns_df.loc[:, ["S&P 500 close"]].copy()

    # Set the short window and long windows
    short_window = 50
    long_window = 100

    # Generate the short and long exponential moving averages (50 and 100 days, respectively)
    sp500_signals["EWM50"] = sp500_signals['S&P 500 close'].ewm(span=short_window).mean()
    sp500_signals["EWM100"] = sp500_signals['S&P 500 close'].ewm(span=long_window).mean()
    sp500_signals["EMA Signal"] = 0.0

    # Generate the trading signal 0 or 1,
    # where 0 is when the EWM50 is under the EWM100, and
    # where 1 is when the EWM50 is higher (or crosses over) the SMA100
    sp500_signals["EMA Signal"][short_window:] = np.where(
    sp500_signals["EWM50"][short_window:] > sp500_signals["EWM100"][short_window:], 1.0, 0.0
)
    # Calculate the points in time at which a position should be taken, 1 or -1
    sp500_signals["Entry/Exit"] = sp500_signals["EMA Signal"].diff()

    return sp500_signals

In [10]:
case_count_signals_df=covid_case_count(case_count_df)

NameError: name 'case_count_df' is not defined

In [11]:
sentiment_signal_df=covid_sentiment(newsapi)
sentiment_signal_df.index = pd.to_datetime(sentiment_signal_df.index)

In [22]:
EMA_signal_df=get_EMA(all_returns_df)

In [23]:
EMA_signal_df.head()

,S&P 500 close,EWM50,EWM100,EMA Signal,Entry/Exit
date,,,,,
2000-06-30,99.2910,99.291000,99.291000,0.0,NaN
2000-07-03,100.6579,99.988119,99.981285,0.0,0.0
2000-07-05,98.8425,99.590871,99.594073,0.0,0.0
2000-07-06,99.6114,99.596315,99.598535,0.0,0.0
2000-07-07,101.2132,99.946069,99.934513,0.0,0.0


In [24]:
def get_trading_signals(sentiment_signal_df,EMA_signal_df):
    #case_count_signals_df=case_count_signals_df.tail(29)
    EMA_signal_df=EMA_signal_df.tail(29)
    trading_signals_df=pd.concat([sentiment_signal_df,EMA_signal_df],axis=1,join="inner")
    trading_signals_df['Signal']=trading_signals_df['Sentiment Signal']+trading_signals_df['EMA Signal']
    trading_signals_df['Entry/Exit']=trading_signals_df['Signal'].diff()
    return trading_signals_df

In [25]:
trading_signals_df=get_trading_signals(sentiment_signal_df,EMA_signal_df)
trading_signals_df.tail()

,Sentiment Signal,Sentiment,S&P 500 close,EWM50,EWM100,EMA Signal,Entry/Exit,Signal
2020-06-25,0.0,Neutral,307.35,298.858814,295.598386,1.0,0.0,1.0
2020-06-26,0.0,Neutral,300.05,298.905527,295.686537,1.0,0.0,1.0
2020-06-29,0.0,Neutral,304.46,299.123349,295.860269,1.0,0.0,1.0
2020-06-30,0.0,Neutral,308.36,299.485571,296.107788,1.0,0.0,1.0
2020-07-01,0.0,Neutral,309.80,299.890058,296.378921,1.0,0.0,1.0


In [26]:
import hvplot.streamz
from streamz import Stream 
from streamz.dataframe import DataFrame

In [34]:
# Initialize Streaming DataFrame for Market Price Data Showing Backtested Signals
data_stream = Stream()
data = all_returns_df['S&P 500 close']
data_updated=pd.DataFrame(data)
data_stream_df = DataFrame(data_stream, example=data_updated)
# Initialize Streaming DataFrame for Signals
#signals_stream = Stream()
#signals_data=trading_signals_df
#signals_stream_df = DataFrame(signals_stream, example=signals_data)

In [40]:
def emit(data): 
    df = pd.DataFrame(data)
    Stream.emit(df)

In [41]:
emit(data)

TypeError: emit() missing 1 required positional argument: 'x'

In [30]:
# Visualize exit position relative to close price
exit = trading_signals_df['Entry/Exit'] == -1.0.hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = trading_signals_df['Entry/Exit'] == 1.0.hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = sp500_signals[['S&P 500 close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = sp500_signals[['EWM50', 'EWM100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)# Visualize exit position relative to close price
exit = sp500_signals[sp500_signals['Entry/Exit'] == -1.0]['S&P 500 close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = sp500_signals[sp500_signals['Entry/Exit'] == 1.0]['S&P 500 close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = sp500_signals[['S&P 500 close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = sp500_signals[['EWM50', 'EWM100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

AttributeError: 'float' object has no attribute 'hvplot'

In [27]:
# Initialize Streaming DataFrame for Market Price Data Showing Backtested Signals
data_stream = Stream()
data = all_returns_df['S&P 500 close']
data_stream_df = DataFrame(data_stream, example=data)
# Initialize Streaming DataFrame for Signals
signals_stream = Stream()
signals_data=trading_signals_df
signals_stream_df = DataFrame(signals_stream, example=signals_data)

TypeError: Streaming DataFrame expects an example of DataFrame like objects. Got: date
2000-06-30     99.2910
2000-07-03    100.6579
2000-07-05     98.8425
2000-07-06     99.6114
2000-07-07    101.2132
                ...   
2020-06-25    307.3500
2020-06-26    300.0500
2020-06-29    304.4600
2020-06-30    308.3600
2020-07-01    309.8000
Name: S&P 500 close, Length: 5029, dtype: float64.

In [45]:
# Visualize exit position relative to close price
exit = sp500_signals[sp500_signals['Entry/Exit'] == -1.0]['S&P 500 close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = sp500_signals[sp500_signals['Entry/Exit'] == 1.0]['S&P 500 close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = sp500_signals[['S&P 500 close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = sp500_signals[['EWM50', 'EWM100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)# Visualize exit position relative to close price
exit = sp500_signals[sp500_signals['Entry/Exit'] == -1.0]['S&P 500 close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = sp500_signals[sp500_signals['Entry/Exit'] == 1.0]['S&P 500 close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = sp500_signals[['S&P 500 close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = sp500_signals[['EWM50', 'EWM100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

NameError: name 'sp500_signals' is not defined

In [46]:
##Dashboard
live_column = pn.Column()
backtesting_column = pn.Column()